# Joint spectral fit on Crab Nebula multi-instrument dataset

## Context
Gammapy allows having multiple datasets of different types (`SpectrumDatasetOnOff`, `MapDataset`) from different instruments.

## Goal of the exercise
Replicate the joint spectral fit from Crab multi-instruments datasets (Fig. 2) as published in [Nigro et al. 2019](https://www.aanda.org/articles/aa/full_html/2019/05/aa34938-18/aa34938-18.html). Datasets are to be used are described in *Datasets* section.

## Datasets
Read the public Crab datasets (DL4) from Nigro et al. 2019 available in `$GAMMAPY_DATA`
Note the data products are stored following the **OGIP format** (originally from the X-ray community) with four different types of files: `BKG`, `ARF`, `PHA` and `RMF` containing the binned counts and corresponding instrument response functions. For more details, see https://gamma-astro-data-formats.readthedocs.io/en/v0.1/ogip/

```
$GAMMAPY_DATA/joint-crab/spectra
├── fact
│   ├── arf_stacked.fits
│   ├── bkg_stacked.fits
│   ├── pha_stacked.fits
│   └── rmf_stacked.fits
├── fermi
│   ├── arf_obs0.fits
│   ├── bkg_obs0.fits
│   ├── pha_obs0.fits
│   └── rmf_obs0.fits
├── hess
│   ├── arf_obs23523.fits
│   ├── bkg_obs23523.fits
│   ├── pha_obs23523.fits
│   ├── rmf_obs23523.fits
│   └── ...
├── magic
│   ├── arf_obs5029747.fits
│   ├── bkg_obs5029747.fits
│   ├── pha_obs5029747.fits
│   ├── rmf_obs5029747.fits
│   └── ...
└── veritas
    ├── arf_obs54809.fits
    ├── bkg_obs54809.fits
    ├── pha_obs54809.fits
    ├── rmf_obs54809.fits
    └── ...
```

In [ ]:
%matplotlib inline

import os
from pathlib import Path
from gammapy.datasets import Datasets, SpectrumDatasetOnOff
from gammapy.modeling.models import LogParabolaSpectralModel, SkyModel, create_crab_spectral_model
import astropy.units as u
from gammapy.modeling import Fit
import matplotlib.pyplot as plt

Open the DL4 datasets of each instrument available in `$GAMMAPY_DATA`

In [ ]:
instruments = ["fermi", "fact", "magic", "veritas", "hess"]

base_path = Path(os.getenv("GAMMAPY_DATA")) / "joint-crab/spectra"
#files_p = list(base_path.rglob("pha*.fits"))

In [ ]:
datasets = Datasets()

for instrument in instruments:
    files = base_path.rglob(f"{instrument}/pha_*.fits")
    for filename in files:
        name=f"_{instrument}_{filename.stem.split('_')[-1]}"
        print(f"Opening DL4 dataset: {name}")
        dataset = SpectrumDatasetOnOff.read(filename).copy(name=name)
        datasets.append(dataset)

Check what `datasets` contains:

In [ ]:
print(datasets)

and also you can access the details of individual datasets:

In [ ]:
datasets[0]

You can also write the datasets back to disk:

In [ ]:
path = Path("datasets/joint_crab/")
path.mkdir(exist_ok=True, parents=True)

filename = path / "datasets_joint_crab_1d.yaml"
datasets.write(filename, overwrite=True)

## Perform joint spectral analysis of all datasets

Steps:
- Define a spectral model and corresponding `SkyModel` and assign it to the datasets.
- Use the `Fit` class to run the joint fit over the datasets.

In [ ]:
spectral_model = LogParabolaSpectralModel(
    amplitude=1e-12 * u.Unit("cm-2 s-1 TeV-1"),
    alpha=2,
    beta=0.05,
    reference=1 * u.TeV,
)
model = SkyModel(spectral_model=spectral_model, name="crab")

datasets.models = [model]

In [ ]:
fit_joint = Fit()
result_joint = fit_joint.run(datasets=datasets)

List the values of resulting best-fit spectral parameters

In [ ]:
spectral_model

Plot the resulting joint fit model (using `sed_type="e2dnde"` and energy range from 10 GeV to 30 TeV) and compare it with spectrum from [Meyer 2010](https://ui.adsabs.harvard.edu/abs/2010A%26A...523A...2M/abstract). 

In [ ]:
crab_meyer = create_crab_spectral_model(reference="meyer")

In [ ]:
fig, ax = plt.subplots()

plot_kwargs = {
    "energy_bounds": [0.01, 30] * u.TeV,
    "sed_type": "e2dnde",
    "ax": ax,
}
spectral_model.plot_error(
    label="Joint fit",
    **plot_kwargs,
)
crab_meyer.plot(
    label="Meyer et al. (2010)",
    **plot_kwargs,
)
ax.legend();